In [83]:
import matplotlib.pyplot as plt
import pandas as pd
from statsmodels.tsa.seasonal import seasonal_decompose
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV
import numpy as np
from datetime import datetime


from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor


from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

## Add failure mode analysis


In [73]:
data = pd.read_csv("../Merging/Merged_Data.csv")

In [74]:

data['period'] = pd.to_datetime(data['period'])
data.sort_values(by='period', inplace=True)
plant_codes = data['plantCode'].unique()
data = data[data['period'] >= '2019-01-01']
pd.set_option('display.max_colwidth', None)

In [75]:
# Define features for training and testing
model_features = ['ELEVATION', 'TEMP',
       'WDSP', 'MXSPD', 'GUST', 'MAX', 'MIN', 'PRCP', 'SNDP','TEMPEXT_BASE40', 'TEMPEXT_BASE45', 'TEMPEXT_BASE50', 
       'OVER_60', 'OVER_70', 'OVER_80', 'UNDER_40', 'UNDER_30', 'UNDER_20',
       'SUM_OVER_UNDER', 'LATITUDE','LONGITUDE','Zip','plantCode']

In [81]:
def train_and_evaluate(model, model_features, data, param_grid=None, test_year=None):
    results = []
    
    # Exclude negative consumption values
    data = data[data['total-consumption'] > 0]

    # Convert 'period' column to datetime format
    data = data.copy()
    data['period'] = pd.to_datetime(data['period'])

    # Extract features and target variable
    X = data[model_features].fillna(0)
    y = data['total-consumption']

    # Define the years for cross-validation folds
    years = sorted(data['period'].dt.year.unique())

    # Iterate over years for cross-validation
    for i in range(len(years)):
        train_years = years[:i+1]  # Include data from previous years
        if test_year == None and i + 1 < len(years):
            test_year = years[i + 1]  # Test on the following year if it exists
        elif test_year != None:
            test_year=test_year
        else:
            break  # No more test years available

        # Filter the data for training and testing
        train_data = data[data['period'].dt.year.isin(train_years)]
        test_data = data[data['period'].dt.year == test_year]

        # Extract features and target variable for training
        X_train = train_data[model_features]
        y_train = train_data['total-consumption']

        # Extract features and target variable for testing
        X_test = test_data[model_features]
        y_test = test_data['total-consumption']

        # Perform hyperparameter tuning using GridSearchCV if it exists
        if param_grid is not None:
            grid_search = GridSearchCV(model, param_grid, cv=5, scoring='neg_mean_squared_error')
            grid_search.fit(X_train, y_train)

            # Get the best model from the grid search
            best_model = grid_search.best_estimator_
            best_params = grid_search.best_params_

            # Make predictions on the test set
            y_pred = best_model.predict(X_test)
        else:
            best_model=model
            best_params = None
            model.fit(X_train, y_train)

            # Make predictions on the test set
            y_pred = model.predict(X_test)

        # Evaluate the model
        mse = mean_squared_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)

        # Store results along with training years and test year
        results.append({'Mean Squared Error': mse, 'R-squared': r2, 'Training Years': train_years, 'Test Year': test_year, 'Best Parameters': best_params})

    results_df = pd.DataFrame(results)
    return results_df

In [80]:
param_grid_rf = {
    'n_estimators': [50, 100, 150],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10]}

# Create a Random Forest model
rf_model = RandomForestRegressor()

# Call the function with the model and other parameters
train_and_evaluate(rf_model, model_features, data, param_grid_rf)

,Mean Squared Error,R-squared,Training Years,Test Year,Best Parameters
0,4.423929e+11,0.824071,[2019],2020,"{'max_depth': 20, 'min_samples_split': 2, 'n_estimators': 100}"
1,5.175337e+11,0.745430,"[2019, 2020]",2021,"{'max_depth': 20, 'min_samples_split': 5, 'n_estimators': 50}"
2,2.164412e+11,0.891452,"[2019, 2020, 2021]",2022,"{'max_depth': 20, 'min_samples_split': 10, 'n_estimators': 150}"
3,4.967570e+11,0.871950,"[2019, 2020, 2021, 2022]",2023,"{'max_depth': 20, 'min_samples_split': 10, 'n_estimators': 150}"


In [78]:
param_grid_knn = {
    'n_neighbors': [3, 5, 7, 9],
    'weights': ['uniform', 'distance'],
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute']}

knn_model = KNeighborsRegressor()

# Call the train_test_and_plot function
train_and_evaluate(knn_model, model_features, data, param_grid_knn)


,Mean Squared Error,R-squared,Training Years,Test Year,Best Parameters
0,2.930567e+11,0.883458,[2019],2020,"{'algorithm': 'auto', 'n_neighbors': 3, 'weights': 'uniform'}"
1,2.258914e+11,0.888886,"[2019, 2020]",2021,"{'algorithm': 'ball_tree', 'n_neighbors': 9, 'weights': 'distance'}"
2,2.074478e+11,0.895963,"[2019, 2020, 2021]",2022,"{'algorithm': 'ball_tree', 'n_neighbors': 9, 'weights': 'distance'}"
3,5.082293e+11,0.868993,"[2019, 2020, 2021, 2022]",2023,"{'algorithm': 'ball_tree', 'n_neighbors': 9, 'weights': 'distance'}"


In [79]:
nb_model = GaussianNB() # Gaussian NB does not have any hyperparameters to tune, unlike the others here.
train_and_evaluate(nb_model, model_features, data)


,Mean Squared Error,R-squared,Training Years,Test Year,Best Parameters
0,7.220006e+11,0.712878,[2019],2020,None
1,5.675818e+11,0.720812,"[2019, 2020]",2021,None
2,5.087411e+11,0.744861,"[2019, 2020, 2021]",2022,None
3,1.047925e+12,0.729874,"[2019, 2020, 2021, 2022]",2023,None


In [84]:
param_grid_rf = {
    'n_estimators': [150],
    'max_depth': [20],
    'min_samples_split': [10]}

# Create a Random Forest model
rf_model = RandomForestRegressor()

# Call the function with the model and other parameters
train_and_evaluate(rf_model, model_features, data, param_grid_rf, test_year=datetime(2023,1,1).year)

,Mean Squared Error,R-squared,Training Years,Test Year,Best Parameters
0,1.161057e+12,0.700712,[2019],2023,"{'max_depth': 20, 'min_samples_split': 10, 'n_estimators': 150}"
1,7.366194e+11,0.810120,"[2019, 2020]",2023,"{'max_depth': 20, 'min_samples_split': 10, 'n_estimators': 150}"
2,6.397999e+11,0.835078,"[2019, 2020, 2021]",2023,"{'max_depth': 20, 'min_samples_split': 10, 'n_estimators': 150}"
3,4.914744e+11,0.873312,"[2019, 2020, 2021, 2022]",2023,"{'max_depth': 20, 'min_samples_split': 10, 'n_estimators': 150}"
4,1.473918e+11,0.962007,"[2019, 2020, 2021, 2022, 2023]",2023,"{'max_depth': 20, 'min_samples_split': 10, 'n_estimators': 150}"


In [85]:
param_grid_knn = {
    'n_neighbors': [9],
    'weights': ['distance'],
    'algorithm': ['ball_tree']}

knn_model = KNeighborsRegressor()

# Call the train_test_and_plot function
train_and_evaluate(knn_model, model_features, data, param_grid_knn, test_year=datetime(2023,1,1).year)

,Mean Squared Error,R-squared,Training Years,Test Year,Best Parameters
0,9.479096e+11,0.755656,[2019],2023,"{'algorithm': 'ball_tree', 'n_neighbors': 9, 'weights': 'distance'}"
1,7.972633e+11,0.794488,"[2019, 2020]",2023,"{'algorithm': 'ball_tree', 'n_neighbors': 9, 'weights': 'distance'}"
2,6.876020e+11,0.822756,"[2019, 2020, 2021]",2023,"{'algorithm': 'ball_tree', 'n_neighbors': 9, 'weights': 'distance'}"
3,5.082293e+11,0.868993,"[2019, 2020, 2021, 2022]",2023,"{'algorithm': 'ball_tree', 'n_neighbors': 9, 'weights': 'distance'}"
4,0.000000e+00,1.000000,"[2019, 2020, 2021, 2022, 2023]",2023,"{'algorithm': 'ball_tree', 'n_neighbors': 9, 'weights': 'distance'}"


In [86]:
nb_model = GaussianNB() # Gaussian NB does not have any hyperparameters to tune, unlike the others here.
train_and_evaluate(nb_model, model_features, data, test_year=datetime(2023,1,1).year)

,Mean Squared Error,R-squared,Training Years,Test Year,Best Parameters
0,2.983311e+12,0.230986,[2019],2023,None
1,2.047261e+12,0.472274,"[2019, 2020]",2023,None
2,1.720580e+12,0.556483,"[2019, 2020, 2021]",2023,None
3,1.047925e+12,0.729874,"[2019, 2020, 2021, 2022]",2023,None
4,5.944781e+09,0.998468,"[2019, 2020, 2021, 2022, 2023]",2023,None
